In [1]:
!pip list

Package                  Version
------------------------ -----------
aiohappyeyeballs         2.6.1
aiohttp                  3.12.13
aiosignal                1.4.0
annotated-types          0.7.0
anyio                    4.9.0
asttokens                3.0.0
attrs                    25.3.0
certifi                  2025.6.15
charset-normalizer       3.4.2
colorama                 0.4.6
comm                     0.2.2
dataclasses-json         0.6.7
debugpy                  1.8.11
decorator                5.2.1
distro                   1.9.0
exceptiongroup           1.3.0
executing                2.2.0
faiss-cpu                1.11.0
filelock                 3.18.0
frozenlist               1.7.0
fsspec                   2025.5.1
graphviz                 0.21
greenlet                 3.2.3
h11                      0.16.0
httpcore                 1.0.9
httpx                    0.28.1
httpx-sse                0.4.1
huggingface-hub          0.33.2
idna                     3.10
importlib_metadat

In [39]:
from typing import TypedDict, List, Annotated
from langchain_core.documents import Document
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
import os
from langgraph.graph import StateGraph
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [3]:
# 환경 변수 설정
load_dotenv()
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")  

# FAISS 벡터 DB 불러오기
embedding_model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large-instruct")
vectorstore = FAISS.load_local("card_QA_faiss_db", embedding_model,allow_dangerous_deserialization=True)


C:\Users\Playdata\AppData\Local\Temp\ipykernel_40004\1982410193.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large-instruct")
c:\Users\Playdata\anaconda3\envs\langgraph_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# 1. State 정의
class GraphState(TypedDict):
    question: Annotated[str, "질문"]
    answer: Annotated[str, "답변"]
    score: Annotated[float, "유사도 점수"]
    retriever_docs: Annotated[List[Document], "유사도 상위문서"]

# 2. 노드 정의
def retriever_node(state: GraphState) -> GraphState:
    docs = vectorstore.similarity_search_with_score(state["question"], k=3)
    retrieved_docs = [doc for doc, _ in docs]
    score = docs[0][1]
    # print("\n[retriever_node] 문서:", [doc.page_content for doc in retrieved_docs])
    print("[queston]:", state["question"])
    print("[retriever_node] 상위 문서의 제목:\n",docs)
    print("[retriever_node] 유사도 점수:", score)
    # print('-'*100)
    # print(docs)
    # print('-'*100)
    # print(retrieved_docs[0])
    return GraphState(score=score, retriever_docs=retrieved_docs)

def grade_documents_node(state: GraphState) -> GraphState:
    return GraphState()

def llm_answer_node(state: GraphState) -> GraphState:
    prompt = ChatPromptTemplate.from_template(
        """
        문서: {docs}
        질문: {question}
        위 문서들을 참고해서 질문에 답변해줘.
        """
    )
    docs_content = "\n---\n".join([doc.page_content for doc in state["retriever_docs"]])
    chain = prompt | ChatOpenAI(model="gpt-4.1-mini-2025-04-14")
    answer = chain.invoke({"docs": docs_content, "question": state["question"]}).content
    print("\n[llm_answer_node] 생성된 답변:", answer)
    return GraphState(answer=answer)

def query_rewrite_node(state: GraphState) -> GraphState:
    prompt = ChatPromptTemplate.from_template(
        """
        원본 질문: {question}
        위 질문의 핵심은 유지하면서, 유사 문서를 더 잘 찾을 수 있도록 질문을 다시 써줘.
        """
    )
    chain = prompt | ChatOpenAI(model="gpt-4.1-mini-2025-04-14")
    new_question = chain.invoke({"question": state["question"]}).content
    print("\n[query_rewrite_node] :", new_question)
    return GraphState(question=new_question)

# 3. 노드 분기 함수 정의
def decide_to_generate(state: GraphState) -> str:
    """
    문서의 유사도 점수에 따라 다음 노드를 결정
    - score가 0.23 이하면 'llm_answer'로 이동
    - score가 0.23 초과면 'query_rewrite'로 이동
    """
    if state["score"] <= 0.23:
        return "llm_answer"
    else:
        return "query_rewrite"


# 4. LangGraph 구성 및 연결
workflow = StateGraph(GraphState)
workflow.add_node("retriever", retriever_node)
workflow.add_node("grade_documents", grade_documents_node)
workflow.add_node("llm_answer", llm_answer_node)
workflow.add_node("query_rewrite", query_rewrite_node)
workflow.set_entry_point("retriever")
workflow.add_edge("retriever", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "llm_answer": "llm_answer",
        "query_rewrite": "query_rewrite",
    },
)
workflow.add_edge("query_rewrite", "retriever")

In [67]:
# 실행 
app = workflow.compile()

response = app.invoke({"question": "비번 변경", "answer": "", "score": 0.0, "retriever_docs": []})
print("\n[최종 답변]:", response["answer"])

[queston]: 비번 변경
[retriever_node] 상위 문서의 제목:
 [(Document(id='50f991c5-7cdc-4fe9-8b6e-288eb0c3b91b', metadata={}, page_content='인증 비밀번호를 3회 잘못 입력 시 앱이 종료되고 누적 5회 시 서비스가 초기화 됩니다. 재가입, 인증 후 이용부탁드립니다.'), np.float32(0.26459503)), (Document(id='6038104b-bd52-4dc6-a759-a4cab6f7b13e', metadata={}, page_content='전체메뉴 > 설정[톱니카드 모양] > 인증 및 보안 > 앱 비밀번호 변경 메뉴를 선택해 재인증/등록으로 앱 비밀번호를 재설정할 수 있습니다. 앱 비밀번호 오류 3회 초과 시 서비스 인증이 해지되어 재인증/등록을 해야 합니다.'), np.float32(0.26668274)), (Document(id='1d55e85d-0f87-4a71-bc2b-e20b8bbd5f23', metadata={}, page_content='등록된 카드를 해제하고 변경하고자 하는 카드 정보를 입력해 등록하시면 됩니다.'), np.float32(0.2671058))]
[retriever_node] 유사도 점수: 0.26459503

[query_rewrite_node] : 비밀번호 변경 방법 알려주세요
[queston]: 비밀번호 변경 방법 알려주세요
[retriever_node] 상위 문서의 제목:
 [(Document(id='67ffa1ec-2c7c-41ce-a5b3-3d9fc8d2c2af', metadata={}, page_content='카드 비밀번호 변경을 원하시는 경우, 다음과 같은 방법으로 변경해주시기 바랍니다. [비밀번호 변경] 하나카드 홈페이지 접속 > 로그인 > 카드 > 발급조회/사용등록 > 카드비밀번호등록/변경 에서 변경 가능합니다. 카드비밀번호등록/변경'), np.float32(0.20978272)), (Document(id='60